# Machine Learning Engineer Nanodegree
## Deep Learning
## Project: Build a Digit Recognition Program

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

----
## Step 1: Design and Test a Model Architecture
Design and implement a deep learning model that learns to recognize sequences of digits. Train the model using synthetic data generated by concatenating character images from [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) or [MNIST](http://yann.lecun.com/exdb/mnist/). To produce a synthetic sequence of digits for testing, you can for example limit yourself to sequences up to five digits, and use five classifiers on top of your deep network. You would have to incorporate an additional ‘blank’ character to account for shorter number sequences.

There are various aspects to consider when thinking about this problem:
- Your model can be derived from a deep neural net or a convolutional network.
- You could experiment sharing or not the weights between the softmax classifiers.
- You can also use a recurrent network in your deep neural net to replace the classification layers and directly emit the sequence of digits one-at-a-time.

You can use ** Keras ** to implement your model. Read more at [keras.io](https://keras.io/).

Here is an example of a [published baseline model on this problem](http://static.googleusercontent.com/media/research.google.com/en//pubs/archive/42241.pdf). ([video](https://www.youtube.com/watch?v=vGPI_JvLoN0)). You are not expected to model your architecture precisely using this model nor get the same performance levels, but this is more to show an exampe of an approach used to solve this particular problem. We encourage you to try out different architectures for yourself and see what works best for you. Here is a useful [forum post](https://discussions.udacity.com/t/goodfellow-et-al-2013-architecture/202363) discussing the architecture as described in the paper.

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

## Implementation Overview


To complete the first step of this project we will need to:

0. [Load required libraries](#Load-required-libraries) and packages
1. [Download data](#Download-data) from the web
2. [Extract data](#Extract-data)
    1. Parse MNIST data format (idx3 file format)
3. [Explore data](#Explore-data):
    1. See how labels are organized
    2. Identify how many labels we have on the training set
    3. See how many labels and training cases we have for each class
    4. Identify potential problems in the dataset
4. [Preprocess data](#Preprocess-data):
    1. Apply transformations if needed to (i.e.: center images, crop, resize, apply grey-scale, etc.)
    2. Remove duplicates
    3. Deal with problems we found during the exploration step
    4. Introduce a "blank" character, if not present.
    5. Save it for future reuse.
5. [Split data](#Split-data) into Train, Validation and Test sets.
6. Create a [batch generator](#Batch-Generator) to feed inputs to our network.
    1. Besides generating batches, this generator has to **concatenate multiple images** in order to generate a sequence of digits / letters
7. [Build model](#Build-Model)(s)
8. [Evaluate model](#Evaluate-Model)(s)
9. [Tune model](#Tune-Model)(s)



---------

[Back to Implementation Overview](#Implementation-Overview)

### Load required libraries

In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
import gzip
from IPython.display import display, Image
from scipy import ndimage
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

//anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


ImportError: No module named idx2numpy

----------

[Back to Implementation Overview](#Implementation-Overview)

### Download data


In [7]:
last_percent_reported = None

def download_progress_hook(count, blockSize, totalSize):
  """A hook to report the progress of a download. This is mostly intended for users with
  slow internet connections. Reports every 5% change in download progress.
  """
  global last_percent_reported
  percent = int(count * blockSize * 100 / totalSize)

  if last_percent_reported != percent:
    if percent % 5 == 0:
      sys.stdout.write("%s%%" % percent)
      sys.stdout.flush()
    else:
      sys.stdout.write(".")
      sys.stdout.flush()
      
    last_percent_reported = percent
        
def maybe_download(url, target_folder, filename, expected_bytes, force=False):
  """Download a file if not present, and make sure it's the right size."""
  global last_percent_reported

  notebook_dir = os.getcwd()
  os.chdir(target_folder)
  if force or not os.path.exists(filename):
    print('Attempting to download:', filename)
    last_percent_reported = None
    filename, _ = urlretrieve(url + filename, filename, reporthook=download_progress_hook)
    print('\nDownload Complete!')
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', filename)
  else:
    os.chdir(notebook_dir)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  os.chdir(notebook_dir)
  return filename

In [13]:
DATASETS_DIR = '/input'
NOTMNIST_URL = 'http://commondatastorage.googleapis.com/books1000/'
MNIST_URL = 'http://yann.lecun.com/exdb/mnist/'

nmnist_train_filename = maybe_download(NOTMNIST_URL, DATASETS_DIR,'notMNIST_large.tar.gz', 247336696)
nmnist_test_filename = maybe_download(NOTMNIST_URL, DATASETS_DIR, 'notMNIST_small.tar.gz', 8458043)
'''b
train-images-idx3-ubyte.gz:  training set images (9912422 bytes) 
train-labels-idx1-ubyte.gz:  training set labels (28881 bytes) 
t10k-images-idx3-ubyte.gz:   test set images (1648877 bytes) 
t10k-labels-idx1-ubyte.gz:   test set labels (4542 bytes)
'''

mnist_train_images_filename = maybe_download(MNIST_URL, DATASETS_DIR, 'train-images-idx3-ubyte.gz', 9912422)
mnist_train_labels_filename = maybe_download(MNIST_URL, DATASETS_DIR, 'train-labels-idx1-ubyte.gz', 28881)
mnist_test_images_filename = maybe_download(MNIST_URL, DATASETS_DIR, 't10k-images-idx3-ubyte.gz', 1648877)
mnist_test_labels_filename = maybe_download(MNIST_URL, DATASETS_DIR, 't10k-labels-idx1-ubyte.gz', 4542)

Found and verified notMNIST_large.tar.gz
Found and verified notMNIST_small.tar.gz
Attempting to download: train-images-idx3-ubyte.gz
0%....5%....10%....15%....20%....25%....30%....35%....40%....45%....50%....55%....60%....65%....70%....75%....80%....85%....90%....95%....100%
Download Complete!
Found and verified train-images-idx3-ubyte.gz
Attempting to download: train-labels-idx1-ubyte.gz
0%..85%.
Download Complete!
Found and verified train-labels-idx1-ubyte.gz
Attempting to download: t10k-images-idx3-ubyte.gz
0%....5%....10%....15%....20%....25%....30%....35%....40%....45%....50%....55%....60%....65%....70%....75%....80%....85%....90%....95%....100%
Download Complete!
Found and verified t10k-images-idx3-ubyte.gz
Attempting to download: t10k-labels-idx1-ubyte.gz
0%180%
Download Complete!
Found and verified t10k-labels-idx1-ubyte.gz


----------

[Back to Implementation Overview](#Implementation-Overview)

### Extract data


In [32]:
def maybe_extract_not_mnist(dataset_folder, filename, force=False):
  if not filename.startswith('notMNIST'):
    raise Exception('Expected data file to start with notMNIST. Found %s instead' % filename )
    
  filepath = os.path.join(dataset_folder, filename)
  root = os.path.splitext(os.path.splitext(filepath)[0])[0]  # remove .tar.gz
  if os.path.exists(root) and not force:
    # You may override by setting force=True.
    print('%s already present - Skipping extraction of %s.' % (root, filepath))
  else:
    print('Extracting data for %s. This may take a while. Please wait.' % root)
    tar = tarfile.open(filepath)
    sys.stdout.flush()
    tar.extractall(path=dataset_folder)
    tar.close()

    # checking notMNIST file structure
    num_classes = 10

    data_folders = [
        os.path.join(root, d) for d in sorted(os.listdir(root))
        if os.path.isdir(os.path.join(root, d))]
    if len(data_folders) != num_classes:
        raise Exception(
            'Expected %d folders, one per class. Found %d instead.' % (
            num_classes, len(data_folders)))
    print(data_folders)
    return data_folders

In [41]:
def load_mnist(dataset_folder, filename):
    filepath = os.path.join(dataset_folder, filename)

    with gzip.open(filepath, 'rb') as f:
        file_content = f.read()
    
    print("Loaded %s to memory" % filepath)
    return file_content

In [37]:
# Testing extraction for notMNIST
nmnist_train_folders = maybe_extract_not_mnist(DATASETS_DIR, nmnist_train_filename)
nmnist_test_folders = maybe_extract_not_mnist(DATASETS_DIR, nmnist_test_filename)

/input/notMNIST_large already present - Skipping extraction of /input/notMNIST_large.tar.gz.
/input/notMNIST_small already present - Skipping extraction of /input/notMNIST_small.tar.gz.


In [42]:
mnist_train_images = load_mnist(DATASETS_DIR, mnist_train_images_filename)
mnist_train_labels = load_mnist(DATASETS_DIR, mnist_train_labels_filename)
mnist_test_images = load_mnist(DATASETS_DIR, mnist_test_images_filename)
mnist_test_labels = load_mnist(DATASETS_DIR, mnist_test_labels_filename)

Loaded /input/train-images-idx3-ubyte.gz to memory
Loaded /input/train-labels-idx1-ubyte.gz to memory
Loaded /input/t10k-images-idx3-ubyte.gz to memory
Loaded /input/t10k-labels-idx1-ubyte.gz to memory


----------

[Back to Implementation Overview](#Implementation-Overview)

### Explore data


----------

[Back to Implementation Overview](#Implementation-Overview)

### Preprocess data


----------

[Back to Implementation Overview](#Implementation-Overview)

### Split data


----------

[Back to Implementation Overview](#Implementation-Overview)

### Batch Generator


----------

[Back to Implementation Overview](#Implementation-Overview)

### Build Model


----------

[Back to Implementation Overview](#Implementation-Overview)

### Evaluate Model


----------

[Back to Implementation Overview](#Implementation-Overview)

### Tune Model


### Question 1
_What approach did you take in coming up with a solution to this problem?_

**Answer:** 

### Question 2
_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)_

**Answer:**

### Question 3
_How did you train your model? How did you generate your synthetic dataset?_ Include examples of images from the synthetic data you constructed.

**Answer:**

----
## Step 2: Train a Model on a Realistic Dataset
Once you have settled on a good architecture, you can train your model on real data. In particular, the [Street View House Numbers (SVHN)](http://ufldl.stanford.edu/housenumbers/) dataset is a good large-scale dataset collected from house numbers in Google Street View. Training on this more challenging dataset, where the digits are not neatly lined-up and have various skews, fonts and colors, likely means you have to do some hyperparameter exploration to perform well.

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:


### Your code implementation goes here.
### Feel free to use as many code cells as needed.



### Question 4
_Describe how you set up the training and testing data for your model. How does the model perform on a realistic dataset?_

**Answer:**

### Question 5
_What changes did you have to make, if any, to achieve "good" results? Were there any options you explored that made the results worse?_

**Answer:**

### Question 6
_What were your initial and final results with testing on a realistic dataset? Do you believe your model is doing a good enough job at classifying numbers correctly?_

**Answer:**

----
## Step 3: Test a Model on Newly-Captured Images

Take several pictures of numbers that you find around you (at least five), and run them through your classifier on your computer to produce example results. Alternatively (optionally), you can try using OpenCV / SimpleCV / Pygame to capture live images from a webcam and run those through your classifier.

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:


### Your code implementation goes here.
### Feel free to use as many code cells as needed.



### Question 7
_Choose five candidate images of numbers you took from around you and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult?_

**Answer:**

### Question 8
_Is your model able to perform equally well on captured pictures or a live camera stream when compared to testing on the realistic dataset?_

**Answer:**

### Optional: Question 9
_If necessary, provide documentation for how an interface was built for your model to load and classify newly-acquired images._

**Answer:** Leave blank if you did not complete this part.

----
### Step 4: Explore an Improvement for a Model

There are many things you can do once you have the basic classifier in place. One example would be to also localize where the numbers are on the image. The SVHN dataset provides bounding boxes that you can tune to train a localizer. Train a regression loss to the coordinates of the bounding box, and then test it. 

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:


### Your code implementation goes here.
### Feel free to use as many code cells as needed.



### Question 10
_How well does your model localize numbers on the testing set from the realistic dataset? Do your classification results change at all with localization included?_

**Answer:**

### Question 11
_Test the localization function on the images you captured in **Step 3**. Does the model accurately calculate a bounding box for the numbers in the images you found? If you did not use a graphical interface, you may need to investigate the bounding boxes by hand._ Provide an example of the localization created on a captured image.

**Answer:**

----
## Optional Step 5: Build an Application or Program for a Model
Take your project one step further. If you're interested, look to build an Android application or even a more robust Python program that can interface with input images and display the classified numbers and even the bounding boxes. You can for example try to build an augmented reality app by overlaying your answer on the image like the [Word Lens](https://en.wikipedia.org/wiki/Word_Lens) app does.

Loading a TensorFlow model into a camera app on Android is demonstrated in the [TensorFlow Android demo app](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/examples/android), which you can simply modify.

If you decide to explore this optional route, be sure to document your interface and implementation, along with significant results you find. You can see the additional rubric items that you could be evaluated on by [following this link](https://review.udacity.com/#!/rubrics/413/view).

### Optional Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:


### Your optional code implementation goes here.
### Feel free to use as many code cells as needed.



### Documentation
Provide additional documentation sufficient for detailing the implementation of the Android application or Python program for visualizing the classification of numbers in images. It should be clear how the program or application works. Demonstrations should be provided. 

_Write your documentation here._

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.